## bokehPlot.py

In [27]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.embed import components
from bokeh.io import output_notebook
from bokeh.palettes import Category10

import numpy as np
import pandas as pd
import Picklez

# output_notebook()

qdf = Picklez.load_obj('quandlData2017')
qdf['date'] = qdf['date'].apply(lambda x: pd.to_datetime(x))
qdf = qdf.set_index('date')

def generate_bokeh_plot(qdf,tickers,data_types):
    p1 = figure(title="WIKI/PRICES", toolbar_location="right",background_fill_color="#E8DDCB", x_axis_type="datetime",
                logo=None, tools="pan,wheel_zoom,box_zoom,reset,save")

    for m, ticker in enumerate(tickers):
        for n, data_type in enumerate(data_types):
            idxs=qdf['ticker']==ticker
            p1.line(qdf.index[idxs],  qdf[idxs][data_type], 
                    color=Category10[10][n+(n+1)*m], line_width=2, alpha=0.7, 
                    legend=ticker+':'+data_type)    

    p1.legend.location = "top_left"
    p1.legend.background_fill_color = "darkgrey"
    p1.xaxis.axis_label = 'time'
    p1.yaxis.axis_label = ''
    
    return p1
    
# p = generate_bokeh_plot(qdf,['A','Z'],['open','close','high','low'])
# show(p)


quandlData2017 remembered!


## app.py

In [28]:
from flask import Flask, render_template, request, redirect, flash

app = Flask(__name__)

app.config.from_object('config')

@app.route('/')
@app.route('/index')
def index():
    tickers = np.unique(qdf['ticker'])
    data_types = [key for key in qdf.keys()]
    data_types.pop( data_types.index('ticker') )
    
    # Determine the number of tickers to plot
    nplot = request.args.get("nplot")
    init_config = True
    if nplot == None:
        nplot = 1
    else:
        nplot = int(nplot)
        init_config = False
        
    # Determine which tickers to plot
    tickers2Plot = []
    if init_config:
        tickers2Plot.append(tickers[0])
    else:
        for n in xrange(nplot):
            val = request.args.get("ticker[%d]"%n)
            if val is None:
                tickers2Plot.append(tickers[0])
            else:
                tickers2Plot.append(val)
    print tickers2Plot
        
    # Determine which data types to plot
    data2Plot = []
    if init_config:
        data2Plot.append(data_types[0])
    else:
        for dt in data_types:
            if bool(request.args.get(dt)):
                data2Plot.append(dt)
        
    # Generate the plot
    script, div = components(generate_bokeh_plot(qdf,tickers2Plot,data2Plot))        
    
    return render_template("index.html", nPlotMax=5, nplot=nplot, tickers=tickers, data_types=data_types,
                                        tickers2Plot=tickers2Plot, data2Plot=data2Plot, script=script, div=div)


if __name__ == '__main__':
  app.run(port=33507)

 * Running on http://127.0.0.1:33507/ (Press CTRL+C to quit)


[u'AAON', u'A', u'AAPL']


127.0.0.1 - - [21/Mar/2018 17:38:58] "GET /?nplot=3&ticker%5B0%5D=AAON&ticker%5B1%5D=A&ticker%5B2%5D=AAPL&open=True&ex-dividend=True&split_ratio=True HTTP/1.1" 200 -


[u'AAON', u'A', u'AAPL']


127.0.0.1 - - [21/Mar/2018 17:39:08] "GET /?nplot=3&ticker%5B0%5D=AAON&ticker%5B1%5D=A&ticker%5B2%5D=AAPL&open=True&high=True HTTP/1.1" 200 -


In [10]:
 data_types = [key for key in qdf.keys()]
print data_types
data_types.pop( data_types.index('ticker') )

print data_types

[u'ticker', u'open', u'high', u'low', u'close', u'volume', u'ex-dividend', u'split_ratio', u'adj_open', u'adj_high', u'adj_low', u'adj_close', u'adj_volume']
[u'open', u'high', u'low', u'close', u'volume', u'ex-dividend', u'split_ratio', u'adj_open', u'adj_high', u'adj_low', u'adj_close', u'adj_volume']
